### 1. Purpose of this notebook

The purpose of this notebook is to pre process the data, including removing the data not necessary for the analysis, inclusion of new data and some dtypes transformations.

We removed seven customers from the base. Six had the value of purchases with no value - although there are values in the column MntTotal - and one customer for having more purchases at a discount than the total purchases made.

### 2. Read data

#### 2.1 Import Python packages

In [253]:
import pandas as pd
import datetime

from src.paths import DATA

In [254]:
df = pd.read_csv(DATA / 'ml_project1_data_cleaned.csv')

df = (df.assign(Dt_Customer = pd.to_datetime(df['Dt_Customer'])))

### 3. New columns

For our analysis, we need some new columns:

- The age for each customer.
- The total amount spent in the last two years.
- The total number of purchases in the three channels in the last two years.
- The total of accepteds the offer in the first five campaigns.
- The total of accepteds the offer in the first five campaigns and the pilot campaigns.
- The total of year after registration.
- The total children.

#### Age

In [255]:
current_year = datetime.date.today().year

df = (df
      .assign(Age = current_year - df['Year_Birth'])
     )

#### Total amount spent

In [256]:
df = (df
      .assign(MntTotal = df[['MntWines', 'MntFruits','MntMeatProducts',
                              'MntFishProducts', 'MntSweetProducts']]
                          .sum(axis=1))
     )

#### Total purchases

In [257]:
df = (df
      .assign(NumTotalPurchases = df[['NumWebPurchases','NumCatalogPurchases',
                                     'NumStorePurchases',]]
                                  .sum(axis=1))
     )

#### Total accepted in the first five campaigns

In [258]:
df = (df
      .assign(AcceptedTotalFirstFiveCmps = df[['AcceptedCmp3', 'AcceptedCmp4', 
                                               'AcceptedCmp5', 'AcceptedCmp1', 'AcceptedCmp2']]
                                          .sum(axis=1))
     )

#### Total accepted in the first five campaigns + pilot campaign 

In [259]:
df = (df
      .assign(AcceptedTotalFirstFiveCmpsMorePilot = df[['AcceptedTotalFirstFiveCmps', 'Response']]
                          .sum(axis=1))
     )

#### Total year after registration

In [260]:
df = (df
      .assign(TotalYearAfterRegistration = current_year - df.Dt_Customer.dt.year)
     )

#### Total of childrens

In [261]:
df = (df
      .assign(TotalChildren = df['Kidhome'] + df['Teenhome'])
     )

#### Days of registration

In [262]:
print("data more recent:", df['Dt_Customer'].sort_values(ascending=False).head(1))

now = pd.to_datetime('2014-06-30', format="%Y-%m-%d")

data more recent: 989   2014-06-29
Name: Dt_Customer, dtype: datetime64[ns]


In [263]:
df = (df
      .assign(DaysOfRegistration = (now - pd.to_datetime(df['Dt_Customer'])).dt.days)
             )

#### Total purchases by month

In [274]:
df = (df
      .assign(NumTotalPurchasesByMonthRegistred = df['NumTotalPurchases'] / (df['DaysOfRegistration']/30))
             )

#### Purchase every N days

In [285]:
df =  (df
      .assign(PurchaseEveryNDays = df['DaysOfRegistration'] / df['NumTotalPurchases'])
             )

#### Results

In [266]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
ID,2240.0,5.592160e+03,3246.662198,0.0000,2828.250000,5458.500000,8427.750000,1.119100e+04
Year_Birth,2240.0,1.968806e+03,11.984069,1893.0000,1959.000000,1970.000000,1977.000000,1.996000e+03
Income,2216.0,5.224725e+04,25173.076661,1730.0000,35303.000000,51381.500000,68522.000000,6.666660e+05
Kidhome,2240.0,4.441964e-01,0.538398,0.0000,0.000000,0.000000,1.000000,2.000000e+00
Teenhome,2240.0,5.062500e-01,0.544538,0.0000,0.000000,0.000000,1.000000,2.000000e+00
Recency,2240.0,4.910938e+01,28.962453,0.0000,24.000000,49.000000,74.000000,9.900000e+01
MntWines,2240.0,3.039357e+02,336.597393,0.0000,23.750000,173.500000,504.250000,1.493000e+03
MntFruits,2240.0,2.630223e+01,39.773434,0.0000,1.000000,8.000000,33.000000,1.990000e+02
MntMeatProducts,2240.0,1.669500e+02,225.715373,0.0000,16.000000,67.000000,232.000000,1.725000e+03
MntFishProducts,2240.0,3.752545e+01,54.628979,0.0000,3.000000,12.000000,50.000000,2.590000e+02


There are some customers without purchases (NumTotalPurchases), but with value in MntTotal.
It's necessary remove them, as they may be errors.

In [267]:
df.query('MntTotal > 0 and NumTotalPurchases == 0')['ID']

655      5555
981      3955
1245     6862
1524    11110
1846     9931
2132    11181
Name: ID, dtype: int64

We will also remove cases of customers with more discounts than purchases:

In [268]:
df.query(' NumDealsPurchases > NumTotalPurchases')['ID']

1042    10749
1245     6862
1846     9931
Name: ID, dtype: int64

In [269]:
df.drop(df.query("MntTotal > 0 and NumTotalPurchases == 0").index, inplace=True)
df.drop(df.query("NumDealsPurchases > NumTotalPurchases").index, inplace=True)
df.describe().T

,count,mean,std,min,25%,50%,75%,max
ID,2233.0,5583.114644,3244.053607,0.000000,2815.000000,5453.000000,8418.000000,11191.0
Year_Birth,2233.0,1968.803403,11.984284,1893.000000,1959.000000,1970.000000,1977.000000,1996.0
Income,2209.0,52262.071978,24917.065825,2447.000000,35416.000000,51411.000000,68487.000000,666666.0
Kidhome,2233.0,0.444693,0.538575,0.000000,0.000000,0.000000,1.000000,2.0
Teenhome,2233.0,0.507389,0.544658,0.000000,0.000000,0.000000,1.000000,2.0
Recency,2233.0,49.085087,28.976738,0.000000,24.000000,49.000000,74.000000,99.0
MntWines,2233.0,304.849978,336.724809,0.000000,24.000000,176.000000,505.000000,1493.0
MntFruits,2233.0,26.374384,39.813403,0.000000,2.000000,8.000000,33.000000,199.0
MntMeatProducts,2233.0,167.439767,225.895591,0.000000,16.000000,68.000000,232.000000,1725.0
MntFishProducts,2233.0,37.637707,54.677557,0.000000,3.000000,12.000000,50.000000,259.0


### 4. Remove columns

- The column Z_Revenue is wrong. All the rows containing the same valor: 11. We know that the sucess of the oilot campaign was 15%, not 100%. We created a new column to fix it.

#### Z_Revenue

In [270]:
df = df.drop(columns='Z_Revenue')

### 5. Data transformation

In [271]:
columns = ['AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1',
           'AcceptedCmp2', 'Complain', 'Response']

df[columns] = df[columns].astype(bool)

In [272]:
df.to_csv(DATA / 'ml_project1_data_pre_processed.csv', index=False)